## Лабораторная работа №3

Постановка задачи:

1. Используя тестовые функции для оптимизации из лабораторной работы №1, применить к алгоритму градиентного спуска (GD) компилятор Numba,
2. Сравнить время работы до и после применения Numba.

In [ ]:
import numpy as np
import timeit
from numba import njit

### Функция Матьяса

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
<font size = "3">*Формула:*</font>
<br>
$$
f(x, y) = 0.26 (x^2 + y^2) - 0.48xy,
$$
<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
<font size = "3">*Глобальный минимум:*</font>
<br>
$$
f(0, 0) = 0,
$$
<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
<font size = "3">*Метод поиска:*</font>
<br>
$$
-10\leq x,y\leq 10.
$$
<br>

In [ ]:
def MatyasFunc(x, y):
    return 0.26 * ((x ** 2) + (y ** 2)) - 0.48 * x * y

def GradMatyasFunc(x, y):
    return 0.52 * x - 0.48 * y, -0.48 * x + 0.52 * y

### Функция МакКормика

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
<font size = "3">*Формула:*</font>
<br>
$$
f(x, y) = \sin(x + y) + (x - y)^2 - 1.5x + 2.5y + 1,
$$
<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
<font size = "3">*Глобальный минимум:*</font>
<br><br>
$$
f(-0.54719, -1.54719) = -1.91333,
$$
<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
<font size = "3">*Метод поиска:*</font>
<br>
$$
-1.5\leq x\leq 4,
$$
$$
-3\leq y\leq 4,
$$
<br>

In [ ]:
def McCormickFunc(x, y):
    return np.sin(x + y) + ((x - y) ** 2) - 1.5 * x + 2.5 * y + 1

def GradMcCormickFunc(x, y):
    part1 = 2 * x - 2 * y + np.cos(x + y) - 1.5
    part2 = 2 * y - 2 * x + np.cos(x + y) + 2.5
    return part1, part2 

### Функция Матьяса + Numba

In [ ]:
@njit #alias for @numba.jit(nopython = True)
def numba_MatyasFunc(x, y):
    return 0.26 * ((x ** 2) + (y ** 2)) - 0.48 * x * y

@njit
def numba_GradMatyasFunc(x, y):
    return 0.52 * x - 0.48 * y, -0.48 * x + 0.52 * y

### Функция МакКормика + Numba

In [ ]:
@njit
def numba_McCormickFunc(x, y):
    return np.sin(x + y) + ((x - y) ** 2) - 1.5 * x + 2.5 * y + 1

@njit
def numba_GradMcCormickFunc(x, y):
    part1 = 2 * x - 2 * y + np.cos(x + y) - 1.5
    part2 = 2 * y - 2 * x + np.cos(x + y) + 2.5
    return part1, part2 

### Градиентный спуск

In [ ]:
def GradL(func, g_func, theta0_i, theta1_i, alpha = 0.01, eps = 0.0000001):

    i = 0
    (theta0_i_new, theta1_i_new) = (1000, 1000)

    while (abs(func(theta0_i, theta1_i) - func(theta0_i_new, theta1_i_new)) >= eps):

        if i > 0:
            theta0_i = theta0_i_new
            theta1_i = theta1_i_new

        i += 1

        upd_theta0_i, upd_theta1_i = g_func(theta0_i, theta1_i)
        theta0_i_new = theta0_i - alpha * upd_theta0_i
        theta1_i_new = theta1_i - alpha * upd_theta1_i

    return i, theta0_i, theta1_i, func(theta0_i, theta1_i)

### Градиентный спуск + Numba

In [ ]:
@njit #alias for @numba.jit(nopython = True)
def numba_GradL(func, g_func, theta0_i, theta1_i, alpha = 0.01, eps = 0.0000001):

    i = 0
    (theta0_i_new, theta1_i_new) = (1000, 1000)

    while (abs(func(theta0_i, theta1_i) - func(theta0_i_new, theta1_i_new)) >= eps):

        if i > 0:
            theta0_i = theta0_i_new
            theta1_i = theta1_i_new

        i += 1

        upd_theta0_i, upd_theta1_i = g_func(theta0_i, theta1_i)
        theta0_i_new = theta0_i - alpha * upd_theta0_i
        theta1_i_new = theta1_i - alpha * upd_theta1_i

    return i, theta0_i, theta1_i, func(theta0_i, theta1_i)

### Функция Матьяса. Время выполнения без использования Numba

In [ ]:
x, y = 5, 10

In [ ]:
setup_code = """
from __main__ import GradL, MatyasFunc, GradMatyasFunc, x, y
func = MatyasFunc
g_func = GradMatyasFunc
theta0_i = x
theta1_i = y
"""
print('Time without Numba = ', timeit.timeit(stmt = "GradL(func, g_func, theta0_i, theta1_i)", 
                                             setup = setup_code, 
                                             number = 100))

Time without Numba =  1.3439351449999322


### Функция Матьяса. Время выполнения с использованием Numba

In [ ]:
#stepsGD, x_resGD, y_resGD, resGD = numba_GradL(numba_MatyasFunc, numba_GradMatyasFunc, x, y)
numba_setup_code = """
from __main__ import numba_GradL, numba_MatyasFunc, numba_GradMatyasFunc, x, y
func = numba_MatyasFunc
g_func = numba_GradMatyasFunc
theta0_i = x
theta1_i = y
"""
print('Time with Numba = ', timeit.timeit(stmt = "numba_GradL(func, g_func, theta0_i, theta1_i)", 
                                          setup = numba_setup_code, 
                                          number = 100))

Time with Numba =  0.4547845329998381


### Результат градиентного спуска

In [ ]:
stepsGD, x_resGD, y_resGD, resGD = GradL(MatyasFunc, GradMatyasFunc, x, y)
print('\nGradient descent X result = ' + str(x_resGD) + '\n' + 
      'Gradient descent Y result = ' + str(y_resGD) + '\n' + 
      'Gradient descent function result = ' + str(resGD) + '\n' +
      'Epochs = ' + str(stepsGD) + '\n')


Gradient descent X result = 0.05590078400204313
Gradient descent Y result = 0.05590078400204313
Gradient descent function result = 0.00012499590608172325
Epochs = 7210



### Функция МакКормика. Время выполнения без использования Numba

In [ ]:
x, y = -1, 3

In [ ]:
setup_code = """
from __main__ import GradL, McCormickFunc, GradMcCormickFunc, x, y
func = McCormickFunc
g_func = GradMcCormickFunc
theta0_i = x
theta1_i = y
"""
print('Time without Numba = ', timeit.timeit(stmt = "GradL(func, g_func, theta0_i, theta1_i)", 
                                             setup = setup_code, 
                                             number = 1000))

Time without Numba =  5.479679747999853


### Функция МакКормика. Время выполнения с использованием Numba

In [ ]:
#stepsGD, x_resGD, y_resGD, resGD = numba_GradL(numba_McCormickFunc, numba_GradMcCormickFunc, x, y)
numba_setup_code = """
from __main__ import numba_GradL, numba_McCormickFunc, numba_GradMcCormickFunc, x, y
func = numba_McCormickFunc
g_func = numba_GradMcCormickFunc
theta0_i = x
theta1_i = y
"""
print('Time with Numba = ', timeit.timeit(stmt = "numba_GradL(func, g_func, theta0_i, theta1_i)", 
                                          setup = numba_setup_code, 
                                          number = 1000))

Time with Numba =  0.6663919750001241


### Результат градиентного спуска

In [ ]:
x_real = -0.54719
y_real = -1.54719
f_real = -1.91333

In [ ]:
stepsGD, x_resGD, y_resGD, resGD = GradL(McCormickFunc, GradMcCormickFunc, x, y)
print('\nGradient descent X result = ' + str(x_resGD) + '\n' + 
      'Gradient descent Y result = ' + str(y_resGD) + '\n' + 
      'Gradient descent function result = ' + str(resGD) + '\n' +
      'Epochs = ' + str(stepsGD) + '\n')

diff_x = abs(x_real - x_resGD)
diff_y = abs(y_real - y_resGD)
diff_f = abs(f_real - resGD)
print('Diffrence X result = ' + str(diff_x) + '\n' +
      'Diffrence Y result = ' + str(diff_y) + '\n' +
      'Diffrence function result = ' + str(diff_f) + '\n')


Gradient descent X result = -0.5459181709433332
Gradient descent Y result = -1.5459181698095743
Gradient descent function result = -1.9132201185392779
Epochs = 545

Diffrence X result = 0.0012718290566667845
Diffrence Y result = 0.00127183019042576
Diffrence function result = 0.00010988146072210547

